# **Naive Bayes** (sau Clasificatorul bayesian naiv)

![Grafic](figs/naive-bayes.png "Naive Bayes Classifier")

Este un clasificator probabilistic, bazat pe Teorema lui Thomas Bayes (). Acesta porneste de la premise puternice de independenta intre caracteristicile de intrare $X=(x_{1}, x_{2}, ..., x_{k})$, presupunand fiecare $x_{i}$ ca fiind de sine stator, fara a fi corelat cu altul (de unde vine si numele de "*naiv*").

### Teorema lui Bayes
**Formula**:
$$
\boxed{P(A|B) = \frac{P(B|A)\cdot P(A)}{P(B)}}
$$

**Traducere libera**:

Probabilitatea unui eveniment A, avand deja un eveniment B, poate fi calculata prin probabilitatea evenimentului B, avandu-l pe A, inmultita cu probabilitatea lui A, supra probabilitatea lui B.

### In Machine Learning
Teorema lui Bayes aplicata pentru clasificare este folosita astfel:
$$
\longrightarrow P(y|X)=\frac{P(X|y)\cdot P(y)}{P(X)},
$$
unde: $X=(x_{1}, x_{2}, ..., x_{k})$.

**Traducere libera**:
- $P(y|X)$ => probabilitatea output-ului y daca avem caracteristicile X, numit si **posterior**!!
- $P(X|y)$ => probabilitatea input-ului X daca avem output-ul y
- $P(y)$ => probabilitatea output-ului y
- $P(X)$ => probabilitatea input-ului X

Pornind de la idea ca toate variabilele de intrare $x_{i}$ sunt independente una fata de cealalta, putem separa probabilitatea mare, a lui X in raport cu y, astfel:
$$
P(y|X)=\frac{P(x_{1}|y)\cdot P(x_{2}|y)\cdot ...\cdot P(x_{k}|y)\cdot P(y)}{P(X)}
$$
Intentia este de a selecta clasa $y_{i}$ cu cea mai mare probabilitate, avand caracteristicile $X$. Astfel, se scrie:
$$
y = argmax_{y}(P(y|X)) = argmax_{y}\frac{P(x_{1}|y)\cdot P(x_{2}|y)\cdot ...\cdot P(x_{k}|y)\cdot P(y)}{P(X)}
$$
Avand in vedere ca $P(X)$ nu are legatura cu $y$, il putem neglija, ajungand la ecuatia:
$$
y = argmax_{y}(P(x_{1}|y)\cdot P(x_{2}|y)\cdot ...\cdot P(x_{k}|y)\cdot P(y))
$$
Datorita faptului ca toate probabilitatile apartin $(0,1)$, vor aparea erori de calcul, inmultind numere flotante extrem de mici. Ca sa evitam operatia de inmultire, aplicam logaritmului natural, care are proprietatea de a desface produsul in suma:
$$
\boxed{y = argmax_{y}(ln(P(x_{1}|y)) + ln(P(x_{2}|y)) + ... + ln(P(x_{k}|y)) + ln(P(y)))}
$$

**Cum calculam probabilitatile acestea?**
- $P(y)$ - numita si probabilitatea anterioara, se refera la **frecventa fiecarei** clase (cati de 1 sau de 0 avem, cate mail-uri sunt spam si cate nu)
- $P(x_{i}|y)$ - numita si probabilitatea conditionala a clasei, se calculeaza dupa modelul gausian:
$$
P(x_{i}|y) = \frac{1}{\sqrt{2\pi\sigma_{y}^{2}}}\cdot e^{-(x_{i} - \mu_{y})^{2}/2\sigma_{y}^{2}}
$$
unde:
- $\mu_{y}$ => ***media*** aritmetica a caracteristicilor $x_{i}$ din clasa $y$.
$$
\mu_{y} = \frac{1}{N}\sum_{i=1}^{N}x_{i}
$$
- $\sigma_{y}^{2}$ => ***variatia*** caracteristicii $x_{i}$ din clasa $y$. Expune cat de mult variaza $x_{i}$.
$$
\sigma_{y}^{2}=\frac{1}{N}\sum_{i=1}^{N}(x_{i} - \mu_{y})^{2}
$$
- $\sigma_{y}$ => ***deviatia standard*** a caracteristicii $x_{i}$ din clasa $y$. Expune cat de tare se raspandeste $x_{i}$ fata de media caracteristicilor $x_{i}$.
$$
\sigma_{y}=\sqrt{\frac{1}{N}\sum_{i=1}^{N}(x_{i} - \mu_{y})^{2}}
$$

😵‍💫😵‍💫😵‍💫

### **Algoritm**
**Antrenare:**
- calculam media
- calculam variatia
- calculam probabilitatea anterioara a lui y, aka, frecventa fiecarei clase din dataset.

**Predictie:**
- calculam probabilitatea anterioara (folosim formula lui Gauss pentru probabilitati).
- alegem clasa cu cea mai mare probabilitate posterioara.

-----

### **Studiu de caz**: Supravietuitorii de pe Titanic.
Dorim sa observam care factor a contribuit la supravietuirea dezastrului de pe Titanic din 1912.

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# machine learning algorithm
from sklearn.naive_bayes import GaussianNB

 Începem prin a achiziționa seturile de date de antrenament și de testare în pandas dataframes.

In [ ]:
train_df = pd.read_csv('../datasets/titanic/train.csv')
test_df = pd.read_csv('../datasets/titanic/test.csv')
combine = [train_df, test_df]

### Analizarea datelor
**1. Ce caracteristici sunt in setul de date?**

Observam numele caracteristicilor prezente si valorile lor.

In [ ]:
train_df.head()

In [ ]:
test_df.head()

Ce caracteristici ar trebui sa conteze la supravietuirea accidentului? Ce conteaza?

In [ ]:
train_df.drop([
    'PassengerId',
    'Name',
    'SibSp',
    'Parch',
    'Ticket',
    'Cabin',
    'Embarked',
], axis='columns', inplace=True)

Separam datele de test si cele de antrenament in X si y.

In [ ]:
y_train = train_df['Survived']
X_train = train_df.drop('Survived', axis='columns')

X_test = test_df

X_train.info()

Observam ca, coloana Sex este de tip text, mai precis object. Pentru antrenare insa, avem nevoie de valoare numerica. Asadar vom inlocui 'male'/'female' cu valori numerice. 

In [ ]:
dummy_vals = pd.get_dummies(X_train['Sex'], dtype=int)
dummy_vals.head()

In [ ]:
X_train = pd.concat([X_train, dummy_vals], axis='columns')
X_train.head()

In [ ]:
X_train = X_train.drop('Sex', axis='columns')


Verificam inconsistenta valorilor


In [ ]:
X_train.columns[X_train.isna().any()]

In [ ]:
X_test.isna().sum()

In [ ]:
X_train['Age']

Inlocuim valorile NaN cu media varstelor din dataset.

In [ ]:
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].mean())
X_train['Age']



Datele au fost deja pregatite pentru antrenament si testare. Pregatim deci clasificatorul bayesian naiv.

In [ ]:
print(X_train.shape)
print(y_train.shape)
nb = GaussianNB()
nb.fit(X_train, y_train)

In [ ]:
accuracy = nb.score(X_test)